# License

    Jupyter notebook for accessing CUDA
    Copyright (C) 2018 Andre.Brodtkorb@ifi.uio.no, changed in October by André Brodtkorb

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
#Lets have matplotlib "inline"
%matplotlib inline

#Import packages we need
import numpy as np
import pycuda.compiler as cuda_compiler
from pycuda.gpuarray import GPUArray
import pycuda.driver as cuda_driver

from matplotlib import pyplot as plt

import IPythonMagic

In [2]:
import pytest
from ipytest import run_pytest, clean_tests

In [3]:
from Timer import Timer
import logging

In [4]:
%setup_logging
%cuda_context_handler context

Python version 3.6.6 (default, Sep 12 2018, 18:26:19) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]
Registering context in user workspace
Creating context
PyCUDA version 2018.1.1
CUDA version (9, 1, 0)
Driver version 10000
Using 'Tesla K80' GPU
 => compute capability: (3, 7)
 => memory: 10295 / 11441 MB available
Created context handle <32138384>
Using CUDA cache dir /home/ubuntu/jupyter_notebooks/Simone_Candeloro/MilanoGPU2018/notebooks/cuda_cache


In [5]:
cuda_kernel = """
__global__ void matrixVectorKernel(float* c, float* A, float* b, int a_rows, int a_cols) {
    unsigned int j = blockIdx.x*blockDim.x + threadIdx.x;
    
    //Out of bounds check
    if (j > a_rows) {
        return;
    }
    
    //Compute inner product of row of A with column of B
    float sum = 0.0f;
    for (int i=0; i<a_cols; ++i) {
        unsigned int k = j*a_cols + i;
        sum += A[k] * b[i];
    }
    
    //Write to global memory
    c[j] = sum;
}
"""
module = cuda_compiler.SourceModule(cuda_kernel)
kernel = module.get_function("matrixVectorKernel");

kernel.prepare("PPPiii")

In [6]:
def gpuMatrixVector(a, b):    
    #Create stream
    stream = cuda_driver.Stream()
    
    #Upload data to the device
    #NOTE: We need to make sure that a=(a_rows, a_columns)
    # and that b=(a_colmuns, 1) (column vector)
    # and that c=(a_rows, 1)
    with Timer("Data allocation") as t:
        a_g = GPUArray(a.shape, np.float32)
        b_g = GPUArray(b.shape, np.float32)
        #Allocate output data
        c_g = GPUArray(a.shape[0], np.float32) 
    
    with Timer("A upload") as t:
        a_g.set_async(a, stream=stream)
    with Timer("b upload") as t:
        b_g.set_async(b, stream=stream)
    
    
    #NOTE: We need to change this so that the grid*block is x = 1, y = number of rows in A
    block_size = (256, 1, 1) #These need to be [x, y, z]
    grid_size = (int(np.ceil(a.shape[0] / block_size[0])), 1, 1)

    print("Block size is " + str(block_size))
    print("Grid size is " + str(grid_size))
    
    #Execute program on device
    with Timer("Kernel execution") as t:
        for i in range(100):
            kernel.prepared_async_call(grid_size, block_size, stream, c_g.gpudata, a_g.gpudata, b_g.gpudata, np.int32(a.shape[0]), np.int32(a.shape[1]))     
        
    #Copy data from device to host
    with Timer("Allocate c") as t:
        c = np.empty((a.shape[0], 1), dtype=np.float32)
    with Timer("Download") as t:
        c_g.get(c)
    
    #Return our computed matrix-vector product
    return c

In [7]:
test_size = (2048, 2048)

#Create test input / output data
with Timer("Create test data") as t:
    a = np.random.random(test_size).astype(np.float32)
    b = np.random.random((test_size[1], 1)).astype(np.float32)
with Timer("Run whole function") as t:
    c = gpuMatrixVector(a, b)

Create test data: 77.738047 ms
Data allocation: 0.633240 ms
A upload: 2.783537 ms
b upload: 0.218630 ms
Kernel execution: 0.803471 ms
Run whole function: 7.379532 ms
Exception caught: Resetting to CUDA context context
Traceback (most recent call last):
  File "/home/ubuntu/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3265, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-ab4ffd802ca4>", line 8, in <module>
    c = gpuMatrixVector(a, b)
  File "<ipython-input-6-b2f52f1b417c>", line 31, in gpuMatrixVector
    kernel.prepared_async_call(grid_size, block_size, stream, c_g.gpudata, a_g.gpudata, b_g.gpudata, np.int32(a.shape[0]), np.int32(a.shape[1]))
  File "/home/ubuntu/.local/lib/python3.6/site-packages/pycuda/driver.py", line 511, in function_prepared_async_call
    arg_buf = pack(func.arg_format, *args)
struct.error: pack requires exactly 6 arguments
Popping <32138384>
Pushing <32138384>


Block size is (256, 1, 1)
Grid size is (8, 1, 1)
---------------------------------------------------------------------------
error                                     Traceback (most recent call last)
~/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py in run_code(self, code_obj, result, async_)
   3264                 else:
-> 3265                     exec(code_obj, self.user_global_ns, self.user_ns)
   3266             finally:

<ipython-input-7-ab4ffd802ca4> in <module>
      7 with Timer("Run whole function") as t:
----> 8     c = gpuMatrixVector(a, b)

<ipython-input-6-b2f52f1b417c> in gpuMatrixVector(a, b)
     30         for i in range(100):
---> 31             kernel.prepared_async_call(grid_size, block_size, stream, c_g.gpudata, a_g.gpudata, b_g.gpudata, np.int32(a.shape[0]), np.int32(a.shape[1]))
     32 

~/.local/lib/python3.6/site-packages/pycuda/driver.py in function_prepared_async_call(func, grid, block, stream, *args, **kwargs)
    510         from pyc

Custom TB Handler failed, unregistering


In [ ]:
#Compute reference using Numpy
c_ref = np.dot(a, b)

#Sum of absolute differences
sad = np.sum(np.abs(c - c_ref))

#Print result
# print("C   = ", c)
# print("Ref = ", c_ref)
print("Sad = %.30f" % sad)
print("Per element error: " + str(sad / test_size[1]))

In [ ]:
clean_tests() #initializes pytest
#we now define the tests as "test_name"

def test_gpuMatrixVector():
    #Let us test a matrix of size 1x1
    a = np.ones((1, 1), dtype=np.float32)
    b = 2*np.ones((1, 1), dtype=np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(2.0)
    
    #Test that the inner product works
    a = np.ones((1, 2), dtype=np.float32)
    b = 2*np.ones((2, 1), dtype=np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(4.0)
    
    #Test a general matrix
    test_size = (4, 3)
    a = np.random.random(test_size).astype(np.float32)
    b = np.random.random((test_size[1], 1)).astype(np.float32)
    c = gpuMatrixVector(a, b)
    assert c == pytest.approx(a.dot(b), rel=1e-3)
    
run_pytest(filename='MatrixVectorTesting.ipynb', pytest_options=['-vvv'])